In [1]:
import os
import pandas as pd
import numpy as np
import pickle as pk
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import ccxt
import time
from dotenv import load_dotenv


Bad key "text.kerning_factor" on line 4 in
C:\Users\Josh Bond\Anaconda3\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
load_dotenv()

True

In [3]:
kraken_pub_key = os.getenv("KRAKEN_PUBLIC_KEY")
kraken_sec_key = os.getenv("KRAKEN_SECRET_KEY")
kraken = ccxt.kraken({"apiKey": kraken_pub_key, "secret": kraken_sec_key})

In [17]:
# Getting BTC data and setting columns and date
btc_historical = kraken.fetch_ohlcv("BTC/USD", "1d")
btc_historical_df = pd.DataFrame(
    btc_historical, 
    columns=['date','open','high','low','close','volume']
)
btc_historical_df['date'] = pd.to_datetime(btc_historical_df['date'], unit='ms')
btc_historical_df = btc_historical_df.sort_values(by='date')

In [18]:
start_validation = '2019-04-26'


,date,open,high,low,close,volume
0,2019-04-26,5128.7,5290.0,5043.3,5153.0,4036.150845
1,2019-04-27,5154.5,5216.5,5116.7,5168.5,1650.067797
2,2019-04-28,5169.8,5215.4,5100.2,5158.9,1187.831711
3,2019-04-29,5156.2,5194.2,5061.1,5149.0,1679.761399
4,2019-04-30,5152.8,5297.0,5132.0,5273.0,2809.117861
5,2019-05-01,5272.2,5358.9,5270.0,5324.1,2443.446835
6,2019-05-02,5323.2,5421.7,5311.3,5390.5,2110.269279
7,2019-05-03,5390.0,5796.0,5368.7,5660.0,5333.946731
8,2019-05-04,5658.8,5840.0,5525.0,5763.5,4734.710116
9,2019-05-05,5765.6,5779.9,5627.6,5719.3,1638.092777
